### Money Vs. Happiness

This notebook is used to explore and prep the data for the Streamlit application used in app.py.  

Ideas:
- Q1: General statistics
- Q2: financiallly stable people more satisfied with life?
- Q3: financially conscious people more satisfied with life?

In [20]:
import numpy as np
import pandas as pd

### Load the Data

Reference: [Consumer Financial Well-being](https://www.kaggle.com/srolka/consumer-financial-wellbeing-survey)

In [21]:
data = pd.read_csv("resources/cfwb.csv")

In [22]:
data.head(10)

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.260668
5,10910,1,3,5,7,5,67,5,1,1,...,1,2,3,0,0,0,0,2,0,0.760061
6,11079,1,3,5,4,7,51,1,3,2,...,1,2,4,0,0,1,1,3,0,1.370522
7,7741,1,3,2,2,2,47,1,2,4,...,1,2,4,0,0,0,0,3,0,1.193365
8,8303,1,3,7,7,7,43,4,5,2,...,1,3,7,0,0,0,1,3,0,2.497838
9,9688,1,3,4,5,4,58,3,3,2,...,1,1,2,0,0,0,0,3,0,1.266602


In [23]:
print(len(data))
print(data.columns.values)

6394
['PUF_ID' 'sample' 'fpl' 'SWB_1' 'SWB_2' 'SWB_3' 'FWBscore' 'FWB1_1'
 'FWB1_2' 'FWB1_3' 'FWB1_4' 'FWB1_5' 'FWB1_6' 'FWB2_1' 'FWB2_2' 'FWB2_3'
 'FWB2_4' 'FSscore' 'FS1_1' 'FS1_2' 'FS1_3' 'FS1_4' 'FS1_5' 'FS1_6'
 'FS1_7' 'FS2_1' 'FS2_2' 'FS2_3' 'SUBKNOWL1' 'ACT1_1' 'ACT1_2' 'FINGOALS'
 'PROPPLAN_1' 'PROPPLAN_2' 'PROPPLAN_3' 'PROPPLAN_4' 'MANAGE1_1'
 'MANAGE1_2' 'MANAGE1_3' 'MANAGE1_4' 'SAVEHABIT' 'FRUGALITY' 'AUTOMATED_1'
 'AUTOMATED_2' 'ASK1_1' 'ASK1_2' 'SUBNUMERACY2' 'SUBNUMERACY1'
 'CHANGEABLE' 'GOALCONF' 'LMscore' 'FINKNOWL1' 'FINKNOWL2' 'FINKNOWL3'
 'FK1correct' 'FK2correct' 'FK3correct' 'KHscore' 'KHKNOWL1' 'KHKNOWL2'
 'KHKNOWL3' 'KHKNOWL4' 'KHKNOWL5' 'KHKNOWL6' 'KHKNOWL7' 'KHKNOWL8'
 'KHKNOWL9' 'KH1correct' 'KH2correct' 'KH3correct' 'KH4correct'
 'KH5correct' 'KH6correct' 'KH7correct' 'KH8correct' 'KH9correct'
 'ENDSMEET' 'HOUSING' 'LIVINGARRANGEMENT' 'HOUSERANGES' 'IMPUTATION_FLAG'
 'VALUERANGES' 'MORTGAGE' 'SAVINGSRANGES' 'PRODHAVE_1' 'PRODHAVE_2'
 'PRODHAVE_3' 'PRODHAVE_4'

In [25]:
questions = {

    'Q1': {
        "fpl": "Poverty Status",
        "SWB_1": "I am satisfied with my life",
        "SWB_3": "If i work hard today, I will be more successful in the future",
    },
    
    'Q2': {
        "FWBscore": "Financial well-being scale score",
        "FWB1_1": "I could handle a major unexpected expense",
        "FWB1_3": "Because of my money situation...I will never have things I want in life",
        "FWB1_4": "I can enjoy life because of the way I'm managing my money",
        "FWB1_6": "I am concerned that the money I have or will save won't last",
        "FWB2_1": "Giving a gift would put a strain on my finances for the month",
        "FWB2_2": "I have money left over at the end of the month",
        "FWB2_3": "I am behind with my finances",
        "FWB2_4": "My finances control my life"
    },
    'Q3': {
        "FSscore": "Financial skill scale score",
        "FS1_2": "I know where to find the advice I need to make decisions involving money",
        "FS1_3": "I know how to make complex financial decisions",
        "FS1_4": "I am able to make good financial decisions that are new to me",
        "FS1_5": "I am able to recognize a good financial investment",
        "FS2_1": "I know when I do not have enough info to make a good decision involving my money",
        "FS2_3": "I struggle to understand financial information"
    },
    'Overall': {
        "SUBKNOWL1": "How would you assess your overall financial knowledge"
    }
}